In [1]:
import os
import glob
import psycopg2 as pg
import pandas as pd
from queries import *
%load_ext sql

In [2]:
conn = pg.connect("host=127.0.0.1 dbname=sparkifydb user=postgres password=student")
conn.set_session(autocommit=True)
cur = conn.cursor()

In [3]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files:
            all_files.append(os.path.abspath(f))
    return all_files

In [4]:
song_files = get_files('data/song_data/')

In [5]:
file_path = song_files[0]
print(file_path)

/Users/piyushbhandari/Desktop/data-modelling-postgres/data/song_data/A/A/A/TRAAAEF128F4273421.json


In [6]:
df = pd.read_json(file_path, lines=True)
df.head()

,num_songs,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
0,1,AR7G5I41187FB4CE6C,NaN,NaN,"London, England",Adam Ant,SONHOTT12A8C13493C,Something Girls,233.40363,1982


In [7]:
num_songs, artist_id, artist_latitude, artist_longitude, artist_location, artist_name, song_id, title, duration, year = df.values[0]

In [8]:
artist_data = [artist_id, artist_name, artist_location, artist_longitude, artist_latitude]
artist_data

['AR7G5I41187FB4CE6C', 'Adam Ant', 'London, England', nan, nan]

In [9]:
cur.execute(artist_table_insert, artist_data)

In [10]:
song_data = [song_id, title, artist_id, year, duration] # song_id, title, artist_id, year, duration
song_data

['SONHOTT12A8C13493C',
 'Something Girls',
 'AR7G5I41187FB4CE6C',
 1982,
 233.40363]

In [11]:
cur.execute(song_table_insert, song_data)

# Log Files

In [19]:
log_files = get_files('data/log_data/')

In [20]:
filepath = log_files[0]
print(filepath)

/Users/piyushbhandari/Desktop/data-modelling-postgres/data/log_data/2018/11/2018-11-11-events.json


In [21]:
log_df = pd.read_json(file_path, lines=True)
print(log_df.size)
log_df.head()

10


,num_songs,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
0,1,AR7G5I41187FB4CE6C,NaN,NaN,"London, England",Adam Ant,SONHOTT12A8C13493C,Something Girls,233.40363,1982


In [12]:
# conn.close()
# cur.close()